In [118]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
import time
import datetime
from sklearn import preprocessing
from sklearn import cluster, metrics

In [17]:
rscaler = preprocessing.RobustScaler()

In [4]:
df = pd.read_csv('df.csv')
df.shape

(5929, 7)

In [196]:
def feat_eng(x):
    #build the model x
    y = pd.DataFrame(index = x['customer_unique_id'].unique())

    #feature0  = number of order
    y['n_order'] = x.groupby('customer_unique_id').order_id.count()

    #feature1  = total value of orders
    #y['payment_total'] = x.groupby('customer_unique_id').order_total.agg('sum')

    #feature2-3  = mean and std of orders
    y['payment_mean'] = x.groupby('customer_unique_id').order_total.agg('mean')
    y['payment_std'] = x.groupby('customer_unique_id').order_total.agg('std')

    #feature frequency4-10 of order on the distrib
    x['order_dis'] = pd.cut(x['order_total'],30, duplicates = 'drop')
    temp = pd.crosstab(x['customer_unique_id'], x['order_dis'], values = x['order_dis'], aggfunc= 'count', margins = True)
    for i in range(temp.shape[1]):
        temp.iloc[:,i] = round((temp.iloc[:,i]/temp.iloc[:,-1])*100,2)

    temp = temp.drop(labels = 'All')
    temp = temp.drop(labels = 'All', axis = 1)
    y = y.join(temp)
    #feature frequency11-135 of category
    temp = pd.crosstab(x['customer_unique_id'], x['product_category_name'], values = x['product_category_name'], aggfunc= 'count', margins = True).fillna(0)
    for i in range(temp.shape[1]):
        temp.iloc[:,i] = round((temp.iloc[:,i]/temp.iloc[:,-1])*100,2)
    temp = temp.drop(labels = 'All')
    temp = temp.drop(labels = 'All', axis = 1)
    y = y.join(temp)

    #feature136-137 review score mean and std
    y['rev_mean'] = x.groupby('customer_unique_id').review_score.agg('mean')
    y['rev_std'] = x.groupby('customer_unique_id').review_score.agg('std')
    #feature138  = days since last order
    y['days_since_last_order'] = x.groupby('customer_unique_id').days_since_order.agg('min')
    #feature139 = "age" of the custumer
    y['age'] = x.groupby('customer_unique_id').days_since_order.agg('max')
    y = y.loc[y['n_order'] > 1]
    return y

In [186]:
def simulate(x):
    start = x['days_since_order'].max()
    end = x['days_since_order'].min()
    year = start - 365
    t0 = x.loc[x['days_since_order'] < year ]
    t0 = feat_eng(t0)
    t0s = rscaler.fit_transform(t0.values)
    clus = cluster.KMeans(n_clusters = 30,random_state=38)
    clus.fit(t0s)
    ari_l = []
    year_n = year
    for i in range(int((year-end)/7)):
        tn = x.loc[x['days_since_order'] < year_n]
        tn = feat_eng(tn)
        tns = rscaler.fit_transform(tn.values)
        clus2 = cluster.KMeans(n_clusters = 30,random_state=38)
        tn['pred_labels'] = clus2.fit_predict(tns)
        tn['true_labels'] = clus.predict(tns)
        ari_l.append(metrics.adjusted_rand_score(np.array(tn['true_labels']), np.array(tn['pred_labels'])))
        year_n = year_n+7
    return ari_l

In [165]:
a = ['0','1','2','3']
b = ['1','2']
prout = [x for x in b + a if x not in b or x not in a]
prout

['0', '3']

In [170]:
type(list(t0.columns))

list

In [187]:
test = simulate(df)

ValueError: X has 144 features, but KMeans is expecting 143 features as input.

In [183]:
test

[1.0,
 0.656374685827508,
 0.6064111337825762,
 0.6203245203735714,
 0.7492965710530859,
 0.6689597002691922,
 0.6024660266083472,
 0.6958310772332513,
 0.6853750684771488,
 0.5873695116522157,
 0.6921543902503632,
 0.7188212059620668,
 0.5926382861827043,
 0.6024729206324954,
 0.6177666766872831,
 0.6374694203644474,
 0.6436644987301812,
 0.6240728777505888,
 0.6189945192619432,
 0.6294197567849525,
 0.6760028495108734,
 0.6101059121178811,
 0.6424743087870853,
 0.6829194352603556,
 0.6254632646966881,
 0.5659872368614667,
 0.6131600229218473,
 0.6145999964415111,
 0.5820727142859331,
 0.5872604282712546,
 0.6131466628678117,
 0.6128931741980895,
 0.6282090938052544,
 0.6259105407859926,
 0.6081711889793595,
 0.5890157279337813,
 0.6045018363042657,
 0.6277921500677737,
 0.6364668551904273,
 0.5980840829464263,
 0.5980840829464263,
 0.5971496208628116,
 0.5971496208628116,
 0.5971496208628116,
 0.5971496208628116,
 0.5971496208628116,
 0.5971496208628116]

In [277]:
x = df
start = x['days_since_order'].max()
end = x['days_since_order'].min()
year = start - 365
t0 = x.loc[x['days_since_order'] < year ]
t0 = feat_eng(t0)
t0_s = rscaler.fit_transform(t0.values)
clus = cluster.KMeans(n_clusters = 30,random_state=38)
clus.fit(t0_s)
true_labels = clus.labels_
t0['true_labels'] = true_labels
ari_l = []
year_n = year
t1 = x.loc[x['days_since_order'] < (year_n+7)]
t1 = feat_eng(t1)
t1_s = rscaler.fit_transform(t1.values)
#clus_n = cluster.KMeans(n_clusters = 30,random_state=38)
t1['true_labels'] = clus.predict(t1_s)
clus2 = cluster.KMeans(n_clusters = 30,random_state=38)
t1['pred_labels'] = clus2.fit_predict(t1_s)
temp = pd.DataFrame(t0['true_labels']).join(t1['pred_labels'])
ari_l.append(metrics.adjusted_rand_score(np.array(t1['pred_labels']),np.array(t1['true_labels'])))


In [278]:
ari_l

[0.6566962990335874]

In [199]:
clus.predict(t0_s)

array([3, 1, 1, ..., 1, 1, 2])

In [200]:
clus.predict(t1_s)

array([3, 1, 1, ..., 1, 1, 2])

In [201]:
clus2.fit_predict(t1_s)

array([0, 3, 3, ..., 3, 3, 3])

In [103]:
prout = pd.DataFrame(t0['labels']).join(t1['labels_p'])
np.array(prout['labels'])

array([ 2, 29,  7, ..., 16, 23, 20])

In [129]:
def expl(md_df,name):
    n_order_m = md_df.describe().iloc[1,0]
    payment_m = md_df.describe().iloc[1,1]
    rev_m_m = md_df.describe().iloc[1,-6]
    age_m = md_df.describe().iloc[1,-3]
    #build the explanatory df
    expl_df = pd.DataFrame()
    #first the frequence of the clusters
    expl_df['freq'] = md_df.groupby(name).n_order.count()
    expl_df['freq'] = round((expl_df['freq']/md_df.shape[0])*100,2)
    #the mean ,ratio and max of the number of order
    expl_df['n_order_mean'] = md_df.groupby(name).n_order.agg('mean')
    expl_df['n_order_ratio'] = round(expl_df['n_order_mean']/n_order_m,2)
    expl_df['n_order_max'] = md_df.groupby(name).n_order.agg('max')
    #the mean and the ratio of the mean of payment
    expl_df['payment_m_m'] = md_df.groupby(name).payment_mean.agg('mean')
    expl_df['payment_m_r'] = round(expl_df['payment_m_m']/payment_m,2)
    #the mean and ratio of the mean of review score
    expl_df['rev_m_m'] = md_df.groupby(name).rev_mean.agg('mean')
    expl_df['rev_ratio'] =round(expl_df['rev_m_m']/rev_m_m,2)
    #the mean and ratio of the age of each cluster
    expl_df['age_m_m'] = md_df.groupby(name).age.agg('mean')
    expl_df['age_ratio'] =round(expl_df['age_m_m']/age_m,2)
    #the top2 payment bucket for each cluster
    temp = pd.DataFrame(md_df.groupby(name).mean())
    temp = temp.iloc[:,3:18]
    temp2 = temp.T

    top1=[]
    top2=[]
    for i in range(temp2.shape[1]):    
        val = list(round(temp2.iloc[:,i].nlargest(2),2))
        cat = list(temp2.iloc[:,i].nlargest(2).index)
        cat_top2 = dict(zip(cat, val))
        cat_list = []
        for y in cat_top2:
            cat_list.append(str(y)+':'+str(cat_top2[y]))
        top1.append(cat_list[0])
        top2.append(cat_list[1])
   
       
    expl_df = expl_df.join(pd.DataFrame({'top1_pay' : top1, 'top2_pay' : top2}))
    #the top2 categories for each cluster
    temp = pd.DataFrame(md_df.groupby(name).mean())
    temp = temp.iloc[:,18:-5]
    temp2 = temp.T

    top1=[]
    top2=[]
    for i in range(temp2.shape[1]):    
        val = list(round(temp2.iloc[:,i].nlargest(2),2))
        cat = list(temp2.iloc[:,i].nlargest(2).index)
        cat_top2 = dict(zip(cat, val))
        cat_list = []
        for y in cat_top2:
            cat_list.append(str(y)+':'+str(cat_top2[y]))
        top1.append(cat_list[0])
        top2.append(cat_list[1])
   
       
    expl_df = expl_df.join(pd.DataFrame({'top1_cat' : top1, 'top2_cat' : top2}))

    return expl_df

In [279]:
lala = pd.DataFrame(expl(t1,'true_labels').sort_values(by='payment_m_m').reset_index())


In [280]:
prout = dict(zip(lala['true_labels'],lala.index))

In [282]:
t1['true_labels'] = t1['true_labels'].replace(prout)


In [270]:
t1['true_labels']

6204c4e582a95b6a350adf6988623bfb    0
d3581765bcf955dcf282b1642c74157c    3
35ecdf6858edc6427223b64804cf028e    3
8004f80e361a5ee23aadb7418a685fc2    3
0697142614905a50813d854fa08b8eaf    2
                                   ..
72e05cde2a8e5d8f08ca3ef3bcfadc63    3
24fc2d6f4c3339820a1a3746a125f8c5    0
47a71b49f98984efd3c62cb213f977e8    3
2fc5e667ccf2a1c87a798be23f1c920a    3
acc4424c209233de519e034152f30304    4
Name: true_labels, Length: 1612, dtype: int32

In [283]:
lala2 = pd.DataFrame(expl(t1,'pred_labels').sort_values(by='payment_m_m').reset_index())


In [272]:
t1['pred_labels']

6204c4e582a95b6a350adf6988623bfb    0
d3581765bcf955dcf282b1642c74157c    3
35ecdf6858edc6427223b64804cf028e    3
8004f80e361a5ee23aadb7418a685fc2    3
0697142614905a50813d854fa08b8eaf    1
                                   ..
72e05cde2a8e5d8f08ca3ef3bcfadc63    3
24fc2d6f4c3339820a1a3746a125f8c5    0
47a71b49f98984efd3c62cb213f977e8    3
2fc5e667ccf2a1c87a798be23f1c920a    3
acc4424c209233de519e034152f30304    3
Name: pred_labels, Length: 1612, dtype: int32

In [284]:
prout2 = dict(zip(lala2['pred_labels'], lala2.index))


In [285]:
t1['pred_labels'] = t1['pred_labels'].replace(prout2)


In [275]:
t1['pred_labels']

6204c4e582a95b6a350adf6988623bfb    0
d3581765bcf955dcf282b1642c74157c    4
35ecdf6858edc6427223b64804cf028e    4
8004f80e361a5ee23aadb7418a685fc2    4
0697142614905a50813d854fa08b8eaf    2
                                   ..
72e05cde2a8e5d8f08ca3ef3bcfadc63    4
24fc2d6f4c3339820a1a3746a125f8c5    0
47a71b49f98984efd3c62cb213f977e8    4
2fc5e667ccf2a1c87a798be23f1c920a    4
acc4424c209233de519e034152f30304    4
Name: pred_labels, Length: 1612, dtype: int32

In [286]:
ari_l = []
ari_l.append(metrics.adjusted_rand_score(np.array(t1['true_labels']),np.array(t1['pred_labels'])))
ari_l

[0.6566962990335874]

In [41]:

df['order_dis'] = pd.cut(x['order_total'],30, duplicates = 'drop')

In [40]:
start = df['days_since_order'].max()
end = df['days_since_order'].min()
print(start, end, int(((start-365)-end)/7))


2254 1554 47


In [6]:
test = df.loc[df['days_since_order'] < (start-365) ]
test.shape

(3896, 7)

In [12]:
md_df = feat_eng(test)

E:\temp\ipykernel_38228\2128623664.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['order_dis'] = pd.cut(x['order_total'],30, duplicates = 'drop')


In [13]:
md_df.shape


(1605, 139)

In [14]:
md_df['n_order'].describe()

count    1605.000000
mean        2.088474
std         0.440639
min         2.000000
25%         2.000000
50%         2.000000
75%         2.000000
max        10.000000
Name: n_order, dtype: float64